In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Lambda,Concatenate
from tensorflow.keras import optimizers, regularizers, applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

import numpy as np
import scipy.io as sio
import scipy.stats as sst
import scipy

import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve,roc_auc_score,average_precision_score

import matplotlib.pyplot as plt

import keras

In [ ]:
## load encode feature, prob and tcp
monitor_data = sio.loadmat('./D1_train_features.mat')
probability = monitor_data['probability']
prediction = monitor_data['prediction'].reshape([-1])
label = monitor_data['label'].reshape([-1])
TF = prediction!=label

In [ ]:
monitor_X = probability
monitor_Y = TF
monitor_tcp = np.zeros(TF.shape)
for i in range(TF.shape[0]):
    monitor_tcp[i]=probability[i,int(label[i])]
print(monitor_X.shape, monitor_Y.shape,monitor_tcp.shape)

In [ ]:
## split data into train and test
monitorX_train,monitorX_test,monitorY_train,monitorY_test = train_test_split(monitor_X,
                                                                             monitor_tcp,
                                                                             test_size=0.1,
                                                                             random_state=42)
print(monitorX_train.shape, monitorX_test.shape)

In [ ]:
for i in range(20):
    ## define monitor model
    X1_input = Input(shape=(43,),name='input')
    h = Dense(100,activation='relu',kernel_initializer='random_uniform',name='dense_1')(X1_input)
    h = Dense(50,activation='relu',kernel_initializer='random_uniform',name='dense_2')(h)
    h = Dropout(0.2,name='dropout_1')(h,training=True)
    output = Dense(1, activation='sigmoid',kernel_initializer='random_uniform',name='output')(h)
    monitor_model = models.Model(X1_input,output,name='monitor_without_image')

    ## compile monitor model using mse loss
    monitor_model.compile(optimizer='Adam',loss='mean_squared_error',metrics=['accuracy'])

    ## train and store history
    history = monitor_model.fit(monitorX_train,monitorY_train,epochs=50,validation_split=0.2,shuffle=True,verbose=0)

    ## store the model_model
    store_name = './emsemble_models/pretrained_D1_ensemble_v'+str(i)+'_new.h5'
    print(store_name)
    models.save_model(monitor_model,store_name)